In [2]:
import numpy as np
import pandas as pd

In [74]:
CURR_SOFR_FIX = 0.0455
FIX_DATE = np.datetime64("2023-03-02")

fomc_dates = [
    np.datetime64("2023-03-23"),
    np.datetime64("2023-05-04"),
    np.datetime64("2023-06-15"),
    np.datetime64("2023-07-27"),
    np.datetime64("2023-09-21"),
    np.datetime64("2023-12-14")
]

start_steps = [25] * len(fomc_dates)


In [49]:
def px_1m_sofr_future(r_obs, r_exp, t0, t, T):

    days_t0_t = (t - t0).astype(float)
    days_t_T = (T - t).astype(float)
    days_t0_T = (T - t0).astype(float)

    # 1m SOFR futures are based on a simple airthimetic average of overnight rates
    r = (r_obs * days_t0_t + r_exp * days_t_T) / days_t0_T

    price = 100 - r*100

    return price

def r_exp_from_1m_sofr(px, r_obs, t0, t,  T):

    days_t0_t = (t - t0).astype(float)
    days_t_T = (T - t).astype(float)
    days_t0_T = (T - t0).astype(float)

    r_impl = (100 - px) / 100
    r_exp = (r_impl * days_t0_T - r_obs * days_t0_t) / days_t_T

    return r_exp

In [78]:
def construct_fomc_curve(fix_date, curr_sofr, fomc_dates, start_steps):

    dates = [fix_date] + fomc_dates

    df = pd.DataFrame(
        index = dates, 
        data = {
            'step': [0] + start_steps,
            'curve': curr_sofr
            }
        )
    
    df['curve'] = df['curve'] + df['step'].cumsum() / 10000

    return df

construct_fomc_curve(FIX_DATE, CURR_SOFR_FIX, fomc_dates, start_steps)

,step,curve
2023-03-02,0,0.0455
2023-03-23,25,0.0480
2023-05-04,25,0.0505
2023-06-15,25,0.0530
2023-07-27,25,0.0555
2023-09-21,25,0.0580
2023-12-14,25,0.0605
